<a href="https://colab.research.google.com/github/katunold/reddit-scrape/blob/master/reddit_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all necessary components
!apt-get update
!pip install selenium
!pip install pandas
!pip install anytree
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
# Adding to path, unncessary if files are moved to /usr/bin
# import sys
# sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu

In [0]:
# Hyperparameters
PAGES = 2 # Number of pages from each subreddit
HTTPS = "https://old.reddit.com/"
SUBREDDIT = ["r/all", "r/funny", "r/jokes", "r/sports"]
# Sorting type, set to hot, new, rising, controversial, top, or gilded
SORT = "top"
# Sorting timespan, set to hour, week, month, year, or all.
# No effect for the first 3 sorting types above.
TIME = "week" 

In [0]:
# Import libraries
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as soup
import pandas as pd
import anytree
from anytree import NodeMixin, Node, RenderTree
from pprint import pprint
# Initialize webdriver, set to headless
options = webdriver.ChromeOptions()
options.add_argument("-no-sandbox")
options.add_argument("-headless")
options.add_argument("-disable-dev-shm-usage")
# Open driver with a website, then get results
d = webdriver.Chrome("chromedriver", options=options)

In [0]:
# Custom anytree node for posts' comments
class NodeCom(NodeMixin):
  def __init__(self, name, attrs=None, parent=None, children=None):
    super(NodeCom, self).__init__()
    self.name = name
    self.attrs = attrs # The reddit data
    self.parent = parent
    if children:
      self.children = children

In [5]:
# Get all posts
posts = []
for sub in SUBREDDIT:
  # Iterate through every subreddit listed
  link = HTTPS + sub + "/" + SORT + "/?t=" + TIME
  print(link)
  d.get(link)
  for i in range(PAGES):
    wdposts = d.find_elements_by_class_name("thing")
    # Iterate through every post found
    for wdpost in wdposts:
      html = wdpost.get_attribute("outerHTML")
      htpost = soup(html, "html.parser").div

      # Get post title
      text = htpost.find("a", class_="title").text
      attrs = htpost.attrs # Get everything else in (in attribute)

      # Append all and create room for comment tree
      attrs["data-score"] = int(attrs["data-score"])
      attrs.update({"text": text, "comments": None})
      posts.append(attrs)

      # Go to next page, cancel if there isn't any
    try:
      next = d.find_element_by_class_name("next-button").get_attribute("outerHTML")
    except NoSuchElementException:
      break # End loop
      page = soup(next, "html.parser").a.attrs["href"]
      print(i+1, page)
      d.get(page)
print("Total posts: ", len(posts))
# Create table using pandas DataFrame, just for preview
pd.DataFrame(posts)[0:4]

https://old.reddit.com/r/all/top/?t=week
https://old.reddit.com/r/funny/top/?t=week
https://old.reddit.com/r/jokes/top/?t=week
https://old.reddit.com/r/sports/top/?t=week
Total posts:  200


,class,id,onclick,data-fullname,data-type,data-gildings,data-whitelist-status,data-author,data-author-fullname,data-subreddit,data-subreddit-prefixed,data-subreddit-fullname,data-subreddit-type,data-timestamp,data-url,data-permalink,data-domain,data-rank,data-comments-count,data-score,data-promoted,data-nsfw,data-spoiler,data-oc,data-num-crossposts,data-context,text,comments,data-kind,data-crosspost-root-title,data-crosspost-root-score,data-crosspost-root-num-comments,data-crosspost-root-author,data-crosspost-root-subreddit,data-crosspost-root-subreddit-prefixed,data-crosspost-root-time,data-target-parent-fullname,data-target-root-fullname,data-target-root-type
0,"[, thing, id-t3_fdemc6, odd, gilded, link, ]",thing_t3_fdemc6,click_thing(this),t3_fdemc6,link,90,all_ads,MECHASCHMECK,t2_xntzx,pics,r/pics,t5_2qh0u,public,1583335842000,https://i.redd.it/jxxn2s6jbok41.jpg,/r/pics/comments/fdemc6/2500_mile_flight_2_hou...,i.redd.it,1,5957,205258,false,false,false,false,23,listing,"2500 mile flight, 2 hour drive, 6 hour hike, a...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[, thing, id-t3_fez718, even, link, ]",thing_t3_fez718,click_thing(this),t3_fez718,link,0,all_ads,KONGKONG12,t2_2830ssui,funny,r/funny,t5_2qh33,public,1583605781000,https://v.redd.it/i8mmhkyhnal41,/r/funny/comments/fez718/prof_of_culture_indeed/,v.redd.it,2,1899,166068,false,false,false,false,36,listing,Prof of culture indeed.,None,video,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[, thing, id-t3_fg0037, odd, gilded, link, ]",thing_t3_fg0037,click_thing(this),t3_fg0037,link,3,all_ads,wer190,t2_152mi9,funny,r/funny,t5_2qh33,public,1583781288000,https://i.redd.it/4zxw341e5pl41.jpg,/r/funny/comments/fg0037/i_had_to_cut_down_a_t...,i.redd.it,3,3341,145353,false,false,false,false,48,listing,I had to cut down a tree in my yard and now I ...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[, thing, id-t3_feonl7, even, gilded, link, ]",thing_t3_feonl7,click_thing(this),t3_feonl7,link,2,all_ads,wrenchingpine,t2_11mgfo,pics,r/pics,t5_2qh0u,public,1583548042000,https://i.redd.it/48obxlctv5l41.jpg,/r/pics/comments/feonl7/half_price_thanks_idiots/,i.redd.it,4,3740,143486,false,false,false,false,17,listing,Half price. Thanks idiots.,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Get everything from a comment HTML
def get_comment(htcom):
  # Get comment and vote count
  text = htcom.find("div", class_="usertext-body").text
  vote = htcom.find("span", class_="unvoted")

  # Deleted comments. Also automatically None if hidden
  if vote == '':
    vote = None
  if vote != None:
    vote = int(vote.attrs["title"])

  attrs = htcom.attrs # Get everything else (in attribute)
  attrs.update({"text": text, "data-score": vote})
  return attrs

# Recursively get comments & create tree by linking nodes
def recursive(soup, parent):
  # End recursion if it's empty
  if not soup:
    return;
  # Soup is another link opened by soup
  htcoms = soup[0].find_all("div", class_="comment", recursive=False)
  # Iterate through every comment found, set to the same parent
  
  for htcom in htcoms:
    attrs = get_comment(htcom)
    # Create node. Won't be replaced in the next loop since what matters is the object, not the variable
    node = NodeCom("a", attrs, parent)
    nxsoup = htcom.select("div.listing")
    recursive(nxsoup, node) # Go deeper
    # Nothing to return, since this procedure just links the comment nodes



In [0]:
# Get all comments from every posts, using recursive scrape-store to differentiate between children & parents
for post in posts:
  d.get(HTTPS + post["data-permalink"])
  # Get the raw text page source
  html = d.page_source
  # Find the div containing the comments list, then get the comments unreursively.
  htlist = soup(html, "html.parser").select("div.nestedlisting")
  parent = NodeCom(attrs["data-fullname"], post) # Set the post as the first node
  recursive(htlist, parent)
  post["comments"] = parent

In [8]:
# Preview tree of a single post
prev = posts[0]
print(RenderTree(prev["comments"]))
print()
for pre, fill, node in RenderTree(prev["comments"]):
  vote = node.attrs["data-score"]
  if vote == None:
    vote = 0
  treestr = u"%s%d" % (pre, vote)
  # Preview the text cropped, and newline removed
  print(treestr.ljust(8), node.attrs["text"][:50].replace('\n', ' '))

├── <__main__.NodeCom object at 0x7f0fca121e10>
│   ├── <__main__.NodeCom object at 0x7f0fca1210b8>
│   ├── <__main__.NodeCom object at 0x7f0fca121ef0>
│   └── <__main__.NodeCom object at 0x7f0fca121eb8>
├── <__main__.NodeCom object at 0x7f0fca121e48>
│   ├── <__main__.NodeCom object at 0x7f0fca121e80>
│   │   └── <__main__.NodeCom object at 0x7f0fca0aa048>
│   └── <__main__.NodeCom object at 0x7f0fca121f60>
├── <__main__.NodeCom object at 0x7f0fca121f98>
│   └── <__main__.NodeCom object at 0x7f0fca0aa0f0>
│       ├── <__main__.NodeCom object at 0x7f0fca0aa0b8>
│       │   ├── <__main__.NodeCom object at 0x7f0fca0aa160>
│       │   │   └── <__main__.NodeCom object at 0x7f0fca0aa208>
│       │   │       └── <__main__.NodeCom object at 0x7f0fca0aa2b0>
│       │   │           └── <__main__.NodeCom object at 0x7f0fca0aa358>
│       │   └── <__main__.NodeCom object at 0x7f0fca0aa240>
│       ├── <__main__.NodeCom object at 0x7f0fca0aa198>
│       ├── <__main__.NodeCom object at 0x7f0fca0aa2

In [9]:
# Using the data
# View a certain amount of posts & their attributes
pd.DataFrame(posts)[0:2]

,class,id,onclick,data-fullname,data-type,data-gildings,data-whitelist-status,data-author,data-author-fullname,data-subreddit,data-subreddit-prefixed,data-subreddit-fullname,data-subreddit-type,data-timestamp,data-url,data-permalink,data-domain,data-rank,data-comments-count,data-score,data-promoted,data-nsfw,data-spoiler,data-oc,data-num-crossposts,data-context,text,comments,data-kind,data-crosspost-root-title,data-crosspost-root-score,data-crosspost-root-num-comments,data-crosspost-root-author,data-crosspost-root-subreddit,data-crosspost-root-subreddit-prefixed,data-crosspost-root-time,data-target-parent-fullname,data-target-root-fullname,data-target-root-type
0,"[, thing, id-t3_fdemc6, odd, gilded, link, ]",thing_t3_fdemc6,click_thing(this),t3_fdemc6,link,90,all_ads,MECHASCHMECK,t2_xntzx,pics,r/pics,t5_2qh0u,public,1583335842000,https://i.redd.it/jxxn2s6jbok41.jpg,/r/pics/comments/fdemc6/2500_mile_flight_2_hou...,i.redd.it,1,5957,205258,false,false,false,false,23,listing,"2500 mile flight, 2 hour drive, 6 hour hike, a...",<__main__.NodeCom object at 0x7f0fcac93a90>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[, thing, id-t3_fez718, even, link, ]",thing_t3_fez718,click_thing(this),t3_fez718,link,0,all_ads,KONGKONG12,t2_2830ssui,funny,r/funny,t5_2qh33,public,1583605781000,https://v.redd.it/i8mmhkyhnal41,/r/funny/comments/fez718/prof_of_culture_indeed/,v.redd.it,2,1899,166068,false,false,false,false,36,listing,Prof of culture indeed.,<__main__.NodeCom object at 0x7f0fc9443d68>,video,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# @title Drive Mount {form-width: "30%"}
FILE = "save.dat" # @param {type: "string"}

# Use pickel for file-variable management
import pickle

# Mount Google Drive as folder in "/drive" COllad folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# Write binary to a file
with open(FILE, 'wb') as f:
  pickle.dump(posts, f)
copy_cmd = " "+FILE+" /content/drive/My\ Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
# Test load data, do it after the drive is mounted
copy_cmd = "/content/drive/My\ Drive/"+FILE+" "+"/content/"
!cp $copy_cmd
with open(FILE, 'rb') as f:
  loaded = pickle.load(f)

cp: cannot stat '/content/drive/My Drive/save.dat': No such file or directory


In [16]:
# Preview tree of loaded post.
prev = loaded[1]
print(RenderTree(prev["comments"]))
print()
for pre, fill, node in RenderTree(prev["comments"]):
  vote = node.attrs["votes"]
  if vote == None:
    vote = 0
  treestr = u"%s%d" % (pre, vote)
  print(treestr.ljust(8), node.attrs["text"][:50].replace('\n', ' '))

├── <__main__.NodeCom object at 0x7f0fc9d225f8>
│   ├── <__main__.NodeCom object at 0x7f0fc9d22da0>
│   │   ├── <__main__.NodeCom object at 0x7f0fc9d22668>
│   │   │   └── <__main__.NodeCom object at 0x7f0fca611f60>
│   │   │       ├── <__main__.NodeCom object at 0x7f0fca6115c0>
│   │   │       │   ├── <__main__.NodeCom object at 0x7f0fca4e7978>
│   │   │       │   │   ├── <__main__.NodeCom object at 0x7f0fc9e9d940>
│   │   │       │   │   │   └── <__main__.NodeCom object at 0x7f0fc9d7f978>
│   │   │       │   │   │       └── <__main__.NodeCom object at 0x7f0fc9d7f0b8>
│   │   │       │   │   └── <__main__.NodeCom object at 0x7f0fc9208b38>
│   │   │       │   └── <__main__.NodeCom object at 0x7f0fc7197dd8>
│   │   │       ├── <__main__.NodeCom object at 0x7f0fc7197c50>
│   │   │       ├── <__main__.NodeCom object at 0x7f0fc710dcf8>
│   │   │       ├── <__main__.NodeCom object at 0x7f0fc5cbb5c0>
│   │   │       ├── <__main__.NodeCom object at 0x7f0fc5cbb588>
│   │   │       ├── <__main_

KeyError: ignored